# 처음에는 미세조정을 하면 무조건 성능이 좋아질 것이라 생각했다.
하지만 복잡도가 높아진 탓인지 오히려 성능이 떨어졌다.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 로드 및 전처리
data = pd.read_csv('diabetes.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, 1)

# CNN 모델 구현
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(2, 1), activation='relu', input_shape=(X_train.shape[1], 1, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 1)))
model.add(layers.BatchNormalization())  # 배치 정규화 추가
model.add(layers.Conv2D(64, kernel_size=(2, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 1)))
model.add(layers.BatchNormalization())  # 배치 정규화 추가
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))  # 드롭아웃 비율 조정
model.add(layers.Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'테스트 정확도: {accuracy:.4f}')

# 모델 저장
model.save('diabetes_cnn_model.h5')

# 사전학습 모델 불러오기 및 추가층 추가
base_model = models.load_model('diabetes_cnn_model.h5')
new_model = models.Sequential()
for layer in base_model.layers:
    new_model.add(layer)

new_model.add(layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
new_model.add(layers.Dropout(0.3))  # 드롭아웃 추가
new_model.add(layers.Dense(1, activation='sigmoid'))

# 동결할 층 지정
for layer in new_model.layers[:-2]:
    layer.trainable = False

# 미세조정 및 평가
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # 학습률 조정
                  loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
new_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# 평가
loss, accuracy = new_model.evaluate(X_test, y_test)
print(f'테스트 정확도: {accuracy:.4f}')


Epoch 1/20


C:\Users\tyra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5154 - loss: 0.8802 - val_accuracy: 0.7338 - val_loss: 0.6495
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7092 - loss: 0.5723 - val_accuracy: 0.6883 - val_loss: 0.6171
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7084 - loss: 0.5784 - val_accuracy: 0.6494 - val_loss: 0.6106
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7364 - loss: 0.5179 - val_accuracy: 0.6948 - val_loss: 0.5987
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7239 - loss: 0.5598 - val_accuracy: 0.7013 - val_loss: 0.5900
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7284 - loss: 0.5234 - val_accuracy: 0.7078 - val_loss: 0.5844
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7743 - loss: 0.4923 - val_accuracy: 0.7013 - val_loss: 0.5764
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8009 - loss: 0.4630 - val_accuracy: 0.7013 - val_loss: 0.5693
Ep

테스트 정확도: 0.7403


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3829 - loss: 0.6818 - val_accuracy: 0.3571 - val_loss: 0.6983
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3368 - loss: 0.6918 - val_accuracy: 0.3571 - val_loss: 0.6979
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3673 - loss: 0.6872 - val_accuracy: 0.3571 - val_loss: 0.6976
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3772 - loss: 0.6870 - val_accuracy: 0.3701 - val_loss: 0.6972
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3616 - loss: 0.6851 - val_accuracy: 0.3831 - val_loss: 0.6969
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3753 - loss: 0.6880 - val_accuracy: 0.3766 - val_loss: 0.6965
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3706 - loss: 0.6867 - val_accuracy: 0.3701 - val_loss: 0.6962
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4238 - loss: 0.6871 - val_accuracy: 0.3701 - val_loss

# 그래서 CNN 층도 낮추고 좀 단수화 시켜서 미세조정이 좀 더 효과가 있게 해보려고 했지만
그럼에도 결과가 조금 나아질 뿐 여전히 미세조정이 큰 효과를 거두지 못하였다.

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# 1. 데이터 로드 및 전처리
data = pd.read_csv('diabetes.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# CNN 입력 형상 조정 (2D로 변경)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)  # (샘플 수, 높이, 너비, 채널)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, 1)

# 2. 단순화된 CNN 모델 구현
model = models.Sequential()
model.add(layers.Conv2D(16, kernel_size=(2, 1), activation='relu', input_shape=(X_train.shape[1], 1, 1)))
model.add(layers.MaxPooling2D(pool_size=(1, 1)))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'테스트 정확도: {accuracy:.4f}')

# 모델 저장
model.save('diabetes_cnn_model.h5')

# 3. 사전학습 모델 불러오기 및 추가층 추가
base_model = models.load_model('diabetes_cnn_model.h5')

# 새로운 모델 구축 (단순한 구조 유지)
new_model = models.Sequential()
for layer in base_model.layers:
    new_model.add(layer)

# 추가층과 출력층 추가
new_model.add(layers.Dense(8, activation='relu'))  # 작은 Dense 레이어 추가
new_model.add(layers.Dense(1, activation='sigmoid'))

# 동결할 층 지정
for layer in new_model.layers[:-2]:  # 마지막 두 층 제외
    layer.trainable = False

# 4. 미세조정 및 평가
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 미세조정
new_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# 평가
loss, accuracy = new_model.evaluate(X_test, y_test)
print(f'테스트 정확도: {accuracy:.4f}')


C:\Users\tyra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4549 - loss: 0.6985 - val_accuracy: 0.6299 - val_loss: 0.6529
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7026 - loss: 0.6252 - val_accuracy: 0.6753 - val_loss: 0.5984
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6992 - loss: 0.5698 - val_accuracy: 0.7403 - val_loss: 0.5523
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7239 - loss: 0.5278 - val_accuracy: 0.7857 - val_loss: 0.5170
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7769 - loss: 0.4951 - val_accuracy: 0.7662 - val_loss: 0.4975
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7841 - loss: 0.4613 - val_accuracy: 0.7662 - val_loss: 0.4886
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7625 - loss: 0.4749 - val_accuracy: 0.7727 - val_loss: 0.4838
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8115 - loss: 0.4227 - val_accuracy: 0.7662 - val_loss:

테스트 정확도: 0.7727
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6361 - loss: 0.7137 - val_accuracy: 0.6429 - val_loss: 0.7054
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6634 - loss: 0.7067 - val_accuracy: 0.6429 - val_loss: 0.7006
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6225 - loss: 0.7093 - val_accuracy: 0.6429 - val_loss: 0.6963
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6767 - loss: 0.6915 - val_accuracy: 0.6429 - val_loss: 0.6921
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6588 - loss: 0.6902 - val_accuracy: 0.6429 - val_loss: 0.6880
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6490 - loss: 0.6891 - val_accuracy: 0.6429 - val_loss: 0.6842
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6526 - loss: 0.6843 - val_accuracy: 0.6429 - val_loss: 0.6805
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6705 - loss: 0.6742 - val_accuracy: 0.

### 그래서 레이어를 Conv1d로 바꾸고 동결층을 4개로 하여 미세 조정 과정에서 새로운 데이터에 맞추어 마지막 층들을 조정하되, 이전의 특징 추출 부분은 유지하도록 하였다. 그리고 수차례 돌려보니 다행히 대부분 미세조정한 부분이 좀 더 좋은 결과가 나왔다.


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# 1. 데이터 로드 및 전처리
data = pd.read_csv('diabetes.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# CNN 입력 형상 조정 (1D로 변경)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # (샘플 수, 길이, 채널)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 2. 단순화된 Conv1D 모델 구현
model = models.Sequential()
model.add(layers.Conv1D(16, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))  # Conv1D 레이어
model.add(layers.MaxPooling1D(pool_size=1))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'테스트 정확도: {accuracy:.4f}')

# 모델 저장
model.save('diabetes_conv1d_model.h5')

# 3. 사전학습 모델 불러오기 및 추가층 추가
base_model = models.load_model('diabetes_conv1d_model.h5')

# 새로운 모델 구축 (단순한 구조 유지)
new_model = models.Sequential()
for layer in base_model.layers:
    new_model.add(layer)

# 추가층과 출력층 추가
new_model.add(layers.Dense(8, activation='relu'))  # 작은 Dense 레이어 추가
new_model.add(layers.Dense(1, activation='sigmoid'))

# 동결할 층 지정
for layer in new_model.layers[:-4]:  # 마지막 네 층 제외
    layer.trainable = False

# 4. 미세조정 및 평가
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 미세조정
new_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# 평가
loss, accuracy = new_model.evaluate(X_test, y_test)
print(f'테스트 정확도: {accuracy:.4f}')


Epoch 1/20


C:\Users\tyra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6319 - loss: 0.6840 - val_accuracy: 0.6364 - val_loss: 0.6485
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6812 - loss: 0.6105 - val_accuracy: 0.7143 - val_loss: 0.5779
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7252 - loss: 0.5496 - val_accuracy: 0.7403 - val_loss: 0.5316
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7877 - loss: 0.4829 - val_accuracy: 0.7468 - val_loss: 0.5068
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7654 - loss: 0.5104 - val_accuracy: 0.7792 - val_loss: 0.4963
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7753 - loss: 0.4818 - val_accuracy: 0.7792 - val_loss: 0.4923
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7752 - loss: 0.4666 - val_accuracy: 0.7792 - val_loss: 0.4912
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7938 - loss: 0.4536 - val_accuracy: 0.7727 - val_loss: 0.4920
Ep

테스트 정확도: 0.7468


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3954 - loss: 0.6790 - val_accuracy: 0.6104 - val_loss: 0.6734
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6437 - loss: 0.6674 - val_accuracy: 0.6688 - val_loss: 0.6660
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7376 - loss: 0.6611 - val_accuracy: 0.6948 - val_loss: 0.6593
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7419 - loss: 0.6601 - val_accuracy: 0.7403 - val_loss: 0.6521
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8022 - loss: 0.6421 - val_accuracy: 0.7403 - val_loss: 0.6462
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7666 - loss: 0.6400 - val_accuracy: 0.7532 - val_loss: 0.6392
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7522 - loss: 0.6449 - val_accuracy: 0.7532 - val_loss: 0.6340
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7814 - loss: 0.6272 - val_accuracy: 0.7662 - val_loss: